# SedonaDB KNN + lonboard

In [1]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [4]:
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=places/type=place/"
)

In [5]:
df.show(10)

┌──────────────┬─────────────┬─────────────┬─────────┬───┬─────────────┬─────────────┬─────────────┐
│      id      ┆   geometry  ┆     bbox    ┆ version ┆ … ┆    phones   ┆    brand    ┆  addresses  │
│     utf8     ┆   geometry  ┆    struct   ┆  int32  ┆   ┆     list    ┆    struct   ┆     list    │
╞══════════════╪═════════════╪═════════════╪═════════╪═══╪═════════════╪═════════════╪═════════════╡
│ 77695c36-8e… ┆ POINT(107.… ┆ {xmin: 107… ┆       2 ┆ … ┆ [+84379842… ┆ {wikidata:… ┆ [{freeform… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ d3b0febd-b9… ┆ POINT(107.… ┆ {xmin: 107… ┆       1 ┆ … ┆ [+84911500… ┆ {wikidata:… ┆ [{freeform… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ aa33cedd-ab… ┆ POINT(107.… ┆ {xmin: 107… ┆       2 ┆ … ┆             ┆ {wikidata:… ┆ [{freeform… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌

In [6]:
df.to_view("overture_places")

In [14]:
sd.sql("""
SELECT 
  names.primary AS name,
  categories.primary AS category,
  geometry,
  addresses[0]['region']
FROM overture_places
WHERE categories.primary = 'coffee_shop'
LIMIT 5;
""").show()

┌────────────────────────────┬─────────────┬───────────────────────────┬───────────────────────────┐
│            name            ┆   category  ┆          geometry         ┆ overture_places.addresses │
│            utf8            ┆     utf8    ┆          geometry         ┆    [Int64(0)][region]…    │
╞════════════════════════════╪═════════════╪═══════════════════════════╪═══════════════════════════╡
│ Genchi - Homestay , Coffe… ┆ coffee_shop ┆ POINT(-103.3581841 -82.7… ┆                           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Café Berakhah              ┆ coffee_shop ┆ POINT(-175.2033246 -21.1… ┆                           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Coffee Post                ┆ coffee_shop ┆ POINT(-175.1999178 -21.1… ┆                           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [21]:
sd.sql("""
SELECT 
  names.primary AS name,
  categories.primary AS category,
  geometry
FROM overture_places
WHERE categories.primary = 'coffee_shop'
  AND ST_KNN(
      geometry, 
      ST_SetSRID(ST_GeomFromWKT('POINT(-97.7431 30.2747)'), 4326), 
      5, 
      false
    )
LIMIT 5;
""").show()

SedonaError: This feature is not implemented: Implementation for st_knn([WkbView(Planar, Some(AuthorityCode { authority: "OGC", code: "CRS84" })), Wkb(Planar, Some(AuthorityCode { authority: "OGC", code: "CRS84" })), Arrow(Int64), Arrow(Boolean)]) was not registered